# Unit Test Notebook for Gripper class functions

In [73]:
import importlib
importlib.reload(g)

<module 'magpie.gripper' from '/home/will/MAGPIE/magpie/prompt_planner/../../magpie/gripper.py'>

In [94]:
import sys
sys.path.append('../../')
import magpie.gripper as g
from magpie.gripper import Gripper
import time
servoport = '/dev/ttyACM1'
G = Gripper(servoport=servoport)

Succeeded to open the port
Succeeded to change the baudrate
Moving speed of dxl ID: 1 set to 100 
Moving speed of dxl ID: 2 set to 100 


In [351]:
G.reset_packet_overload()

Torque enable dxl ID: 1 set to 1 
[RxPacketError] Overload error!
[RxPacketError] Overload error!
Torque enable dxl ID: 2 set to 1 
[RxPacketError] Overload error!


In [66]:
G.close_gripper()

Position of dxl ID: 1 set to 586 
Position of dxl ID: 2 set to 450 


In [65]:
G.reset_parameters()

Moving speed of dxl ID: 1 set to 100 
Moving speed of dxl ID: 2 set to 100 
Position of dxl ID: 1 set to 303 
Position of dxl ID: 2 set to 729 


In [72]:
G.open_gripper()

Position of dxl ID: 1 set to 303 
Position of dxl ID: 2 set to 729 


In [86]:
G.get_aperture(finger='both')

79.40480967920502

In [88]:
G.set_torque(130, finger='both')
G.set_speed(60, finger='both')

Moving speed of dxl ID: 1 set to 60 
Moving speed of dxl ID: 2 set to 60 


In [122]:
desired_dist = 105
G.set_goal_aperture(desired_dist, finger='both', debug=True) #25mm aperture between fingers

position: 308
Position of dxl ID: 1 set to 308 
position: 731
Position of dxl ID: 2 set to 731 


In [121]:
# validate z-offset (10mm aperture on either finger should be 20mm)
zoff = G.aperture_to_z(desired_dist, finger='both', debug=True)
print(f"z-offset: {zoff} mm")

theta: 53.8146318475935, l2: 26.567980728689193, l1: 81.32, y_fingertip: 86.88798072868919
z-offset: 86.88798072868919 mm


In [98]:
G.get_aperture(finger='both')

105.48645436150926

In [119]:
G.Finger1.get_present_position()

293

In [118]:
G.get_aperture(finger='left')

53.9484487042254

In [122]:
G.close_until_contact_force(30, 3.0, finger='left', debug=True)

position: 438
Position of dxl ID: 1 set to 295 
Position of dxl ID: 1 set to 294 
Position of dxl ID: 1 set to 293 
Position of dxl ID: 1 set to 290 
Position of dxl ID: 1 set to 289 
Position of dxl ID: 1 set to 288 
Position of dxl ID: 1 set to 287 
Position of dxl ID: 1 set to 286 
Position of dxl ID: 1 set to 285 
Position of dxl ID: 1 set to 284 
Position of dxl ID: 1 set to 283 
Position of dxl ID: 1 set to 282 
Position of dxl ID: 1 set to 281 
Position of dxl ID: 1 set to 280 
Position of dxl ID: 1 set to 279 
Position of dxl ID: 1 set to 278 
Position of dxl ID: 1 set to 277 
Position of dxl ID: 1 set to 275 
Position of dxl ID: 1 set to 274 
Position of dxl ID: 1 set to 273 
Position of dxl ID: 1 set to 272 
left finger reached stop force: 3.2689470400000005 at distance: 54.252107976649704
left finger reached stop load: 192 at position: 292
Position of dxl ID: 1 set to 292 


In [273]:
G.get_load(finger='left')

0

In [5]:
G.set_compliance(2, 7, finger='left', debug=True)
G.set_compliance(2, 7, finger='right', debug=True)

margin_ax12: 9
flexibility_ax12: 128
margin_ax12: 9
flexibility_ax12: 128


In [238]:
[G.get_aperture(finger='left'), G.get_aperture(finger='right')]

[53.20311759145189, 52.04459477753785]

In [51]:
import numpy as np
finger = 'right'
finger_ax12 = G.Finger1 if finger == 'left' else G.Finger2
latency = 0.0006
delay = 0.0055
stop_force = 0.25
stop_load = G.N_to_load(stop_force)
stop_aperture = 30 # mm
curr_pos = G.get_aperture(finger=finger)
curr_pos = finger_ax12.get_present_position()
stop_ax12 = G.theta_to_position(G.aperture_to_theta(stop_aperture), finger=finger, debug=True)
delta = stop_ax12 - int(curr_pos)
sign = np.sign(delta)

stop_ax12, stop_load

position: 601


(601, 100.18198855127346)

In [39]:
G.set_torque(120, finger='both')

In [42]:
finger_ax12.set_goal_position(728)

Position of dxl ID: 2 set to 728 


In [54]:
curr_pos = finger_ax12.get_present_position()
range(curr_pos, stop_ax12)

range(728, 601)

In [63]:
time.sleep(latency)
for next_pos in range(curr_pos, stop_ax12, sign*1):
    finger_ax12.set_goal_position(next_pos)
    time.sleep(delay)
    time.sleep(delay)
    curr_pos = finger_ax12.get_present_position()
    curr_load = finger_ax12.get_load()
    if curr_load > 1023:
        curr_load = curr_load - 1023
    print(f'curr_load: {curr_load}, curr_pos: {curr_pos}')
    time.sleep(delay)
    time.sleep(latency)
    if curr_load > stop_load:
        force = G.load_to_N(curr_load)
        distance = G.get_aperture(finger=finger)
        print(f'{finger} finger reached stop force: {force} at distance: {distance}')
        print(f'{finger} finger reached stop load: {curr_load} at position: {curr_pos}')
        # todo: set goal_distance_fx class variable, but not really necessary
        finger_ax12.set_goal_position(curr_pos)
        break

Position of dxl ID: 2 set to 642 
curr_load: 0, curr_pos: 728
Position of dxl ID: 2 set to 641 
curr_load: 0, curr_pos: 728
Position of dxl ID: 2 set to 640 
curr_load: 0, curr_pos: 726
Position of dxl ID: 2 set to 639 
curr_load: 0, curr_pos: 725
Position of dxl ID: 2 set to 638 
curr_load: 88, curr_pos: 723
Position of dxl ID: 2 set to 637 
curr_load: 88, curr_pos: 722
Position of dxl ID: 2 set to 636 
curr_load: 88, curr_pos: 719
Position of dxl ID: 2 set to 635 
curr_load: 88, curr_pos: 717
Position of dxl ID: 2 set to 634 
curr_load: 88, curr_pos: 715
Position of dxl ID: 2 set to 633 
curr_load: 88, curr_pos: 712
Position of dxl ID: 2 set to 632 
curr_load: 88, curr_pos: 710
Position of dxl ID: 2 set to 631 
curr_load: 16, curr_pos: 708
Position of dxl ID: 2 set to 630 
curr_load: 16, curr_pos: 706
Position of dxl ID: 2 set to 629 
curr_load: 16, curr_pos: 704
Position of dxl ID: 2 set to 628 
curr_load: 16, curr_pos: 700
Position of dxl ID: 2 set to 627 
curr_load: 16, curr_pos: 

In [62]:
finger_ax12.set_goal_position(728)

Position of dxl ID: 2 set to 728 


In [46]:
G.get_load(finger='right')

0

In [47]:
G.open_gripper()

Position of dxl ID: 1 set to 303 
Position of dxl ID: 2 set to 729 


# Testing Getters and Setters

## Getters

In [27]:
print(G.get_load(finger='both'))

[0, 0]


In [28]:
print(G.get_position(finger='both'))

[526, 511]


In [30]:
print(G.get_temp(finger='both'))

[37, 36]


In [33]:
print(G.get_aperture(finger='both'))

19.986677024361736


In [34]:
# compare with Dylan's code
print(G.get_distance(finger='both'))

24.138556173746146


## Setters

In [91]:
G.load_to_N(40)

0.09888000000000001

In [90]:
G.N_to_load(.1)

40.45832530419774

In [41]:
# 1 N
G.set_force(2.0, finger='both', debug=True)
time.sleep(0.005)
G.get_load(finger='both')

converted load: 0


[0, 0]

In [ ]:
# set distance to 20 mm 
G.set_goal_distance(20, debug=True)
time.sleep(0.5)
print(G.get_distance(finger='both'))
print(G.get_aperture(finger='both'))
print(G.get_position(finger='both'))

In [ ]:
# set compliance (in mm, slope[0-7])
G.set_compliance(5, 4, finger='both', debug=True)
time.sleep(0.005) 
print(G.get_compliance(finger='both'))


In [2]:
# https://platform.openai.com/docs/guides/vision
import openai
import base64
import requests
import os
from PIL import Image
from pillow_heif import register_heif_opener
import torch
from transformers.image_utils import ImageFeatureExtractionMixin

# import open3d as o3d
import numpy as np
# import pyrealsense2 as rs
import matplotlib.pyplot as plt
# import RealSense as real

# # Initialize RS435i connection
# rsc = real.RealSense()
# rsc.initConnection()


In [25]:
api_key = os.getenv("OPENAI_API_KEY")


In [31]:
# Path to your image
image_path = "data_snapshots/rgb_image_ur5.png"
image_path_labeled = 'data_snapshots/owlvit_inference.png'

# Getting the base64 string
base64_image = encode_image(image_path)
base64_image_labeled = encode_image(image_path_labeled)

api_key = os.getenv("OPENAI_API_KEY")

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

text1 = """
You are planning a grasp for a robotic manipulator in a grocery bin. 
You will validate whether the labeled objects are present in the bin and correct the list of labels if there are any incongruities.
For example if a the label in the image is "apple (3)" but the object is a "pear", you will correct the label to "pear (3)" in the format of the provided caption.
The number in the image label corresponds to the index of the object in the list of objects.
You are provided with a photo of the grocery bin, a photo of the indexed and labeled grocery bins, and a list of labels with matching indices.
In your response, first identify any errors in the labels or missing labels. Then, finish your response in the format of the provided caption, with corrections applied.
Make sure to mention the index of the fruit associated with the error.
"""

resp1 = """
I see: ["a photo of a lemon", "a photo of an apple", "a photo of a pear", "a photo of an onion", "a photo of a lime"]
Labels: ["lemon", "apple", "pear", "onion", "lime"]
"""

text2 = """
Please generate a list of labels for this new image of the grocery bin, in the same format.
Are there any new fruits or vegetables?
"""
question = """
No further information required.
Now I am looking at a painting of people looking at a shore.
"""

payload = {
    "model": "gpt-4-vision-preview",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": f"{text1}"
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image_labeled}"
            }
          }]
      },
      # {
      #   "role": "assistant",
      #   "content": [
      #     {
      #       "type": "text",
      #       "text": f"{resp1}"
      #     }]
      # },
      # {
      #   "role": "user",
      #   "content": [
      #     {
      #       "type": "text",
      #       "text": f"{text2}"
      #     },
      #     {
      #       "type": "image_url",
      #       "image_url": {
      #         "url": f"data:image/jpeg;base64,{base64_image_avo}"
      #       }
      #     }]
      # }          
      ],
    "max_tokens": 1000
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

{'id': 'chatcmpl-8q2WxRjlJrltz2XHTlAbPBmXc1HHz', 'object': 'chat.completion', 'created': 1707413475, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 2058, 'completion_tokens': 169, 'total_tokens': 2227}, 'choices': [{'message': {'role': 'assistant', 'content': 'Based on the provided images, the following errors in the labels have been identified:\n\n1. The object labeled as "orange (4)" is actually a red apple.\n2. The item labeled as "orange (5)" is a red apple.\n3. The item labeled as "orange (9)" is a green pear.\n\nGiven this information, the corrected list of labels with matching indices is:\n\n- orange (0)\n- orange (1)\n- orange (2)\n- apple (3)\n- apple (4)\n- orange (6)\n- orange (7)\n- orange (8)\n- pear (9)\n- orange (10)\n- orange (11)\n- orange (12)\n- orange (13)\n- orange (14)\n- orange (15)\n- orange (16)\n- orange (17)\n- orange (18)'}, 'finish_reason': 'stop', 'index': 0}]}


In [32]:
generation = response.json()["choices"][0]['message']['content']
generation

'Based on the provided images, the following errors in the labels have been identified:\n\n1. The object labeled as "orange (4)" is actually a red apple.\n2. The item labeled as "orange (5)" is a red apple.\n3. The item labeled as "orange (9)" is a green pear.\n\nGiven this information, the corrected list of labels with matching indices is:\n\n- orange (0)\n- orange (1)\n- orange (2)\n- apple (3)\n- apple (4)\n- orange (6)\n- orange (7)\n- orange (8)\n- pear (9)\n- orange (10)\n- orange (11)\n- orange (12)\n- orange (13)\n- orange (14)\n- orange (15)\n- orange (16)\n- orange (17)\n- orange (18)'

In [40]:
resp1 = """
Here is an example of a correct response to these images:
Based on the provided images, the following errors in the labels have been identified:
n1. The object labeled as "orange (0)" in the top right is actually a yellow lemon.
n2. The item labeled as "orange (3)" to the left of "orange (0)" is a yellow lemon.
n3. The item labeled as "orange (6)" in the far right is a yellow lemon.
n2. The item labeled as "orange (8)" to the left of "orange (6)" is a yellow lemon.
Given this information, the corrected list of labels with matching indices is:\n\n- lemon (0)\n- orange (1)\n- orange (2)\n- lemon (3)\n- orange (4)\n- orange (5)\n- lemon (6)\n- orange (7)\n- lemon (8)\n- orange (9)\n- orange (10)\n- orange (11)\n- orange (12)\n- orange (13)\n- orange (14)\n- orange (15)\n- orange (16)\n- orange (17)\n- orange (18)
"""

In [41]:
text1 = """
You are planning a grasp for a robotic manipulator in a grocery bin. 
You will validate whether the labeled objects are present in the bin and correct the list of labels if there are any incongruities.
For example if a the label in the image is "apple (3)" but the object is a "pear", you will correct the label to "pear (3)" in the format of the provided caption.
The number in the image label corresponds to the index of the object in the list of objects.
You are provided with a photo of the grocery bin, a photo of the indexed and labeled grocery bins, and a list of labels with matching indices.
In your response, first identify any errors in the labels or missing labels. Then, finish your response in the format of the provided caption, with corrections applied.
Make sure to mention the index of the fruit associated with the error.

Here is an example of a caption:
"""
text1 = text1 + context

In [45]:
text2 = """Now, please identifiy any errors or missing labels in the next image. Provided caption: """
text2 = text2 + context_pear
text2

"Now, please identifiy any errors or missing labels in the next image. Provided caption: \npear (0): ([1003.6107110977173, 109.06752079725266, 1086.157660484314, 173.9412572979927], 'pear')\npear (1): ([823.7149906158447, 91.88447713851929, 982.2424983978271, 239.59284782409668], 'pear')\npear (2): ([573.7592506408691, 218.29054534435272, 660.1446342468262, 311.40706837177277], 'pear')\npear (3): ([620.7485294342041, 263.33383083343506, 804.0051937103271, 411.74015522003174], 'pear')\npear (4): ([953.1515741348267, 304.79566991329193, 1039.001441001892, 410.0387817621231], 'pear')\npear (5): ([624.2333507537842, 260.7258278131485, 803.745927810669, 409.4582122564316], 'pear')\npear (6): ([948.1863212585449, 302.46907353401184, 1031.769733428955, 430.90051531791687], 'pear')\npear (7): ([501.7671251296997, 371.20080292224884, 600.0220441818237, 477.94571578502655], 'pear')\npear (8): ([579.2618751525879, 393.62665593624115, 659.7358512878418, 501.1235100030899], 'pear')\n"

In [50]:
# Path to your image
image_path = "data_snapshots/rgb_image_ur5.png"
image_path_orange = 'data_snapshots/owlvit_inference-orange.png'
image_path_pear = 'data_snapshots/owlvit_inference-pear.png'

# Getting the base64 string
base64_image = encode_image(image_path)
base64_image_orange = encode_image(image_path_orange)
base64_image_pear = encode_image(image_path_pear)

api_key = os.getenv("OPENAI_API_KEY")

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}



# resp1 = """
# I see: ["a photo of a lemon", "a photo of an apple", "a photo of a pear", "a photo of an onion", "a photo of a lime"]
# Labels: ["lemon", "apple", "pear", "onion", "lime"]
# """

text2 = """
Please generate a list of labels for this new image of the grocery bin, in the same format.
Are there any new fruits or vegetables?
"""
question = """
No further information required.
Now I am looking at a painting of people looking at a shore.
"""

payload = {
    "model": "gpt-4-vision-preview",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": f"{text1}"
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image_orange}"
            }
          }
          ]
      },
      {
        "role": "assistant",
        "content": [
          {
            "type": "text",
            "text": f"{resp1}"
          }]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": f"{text2}"
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          }
          ,
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image_pear}"
            }
          }
          ]
      }          
      ],
    "max_tokens": 1000
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

{'id': 'chatcmpl-8q2phOt1556w4tx7WqU92SYBQLnYW', 'object': 'chat.completion', 'created': 1707414637, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 3922, 'completion_tokens': 104, 'total_tokens': 4026}, 'choices': [{'message': {'role': 'assistant', 'content': 'Based on the provided image, here is a list of labels with corrections to the indices and types of fruits they correspond to:\n\n- pear (0)\n- pear (1)\n- pear (2)\n- pear (3)\n- pear (4)\n- pear (5)\n- pear (6)\n- pear (7)\n- pear (8)\n\nThere are no new fruits or vegetables in the image beyond the pears. Each labeled item appears to be a pear, correctly matching the label. No adjustments are needed.'}, 'finish_reason': 'stop', 'index': 0}]}


In [51]:
generation = response.json()["choices"][0]['message']['content']
generation

'Based on the provided image, here is a list of labels with corrections to the indices and types of fruits they correspond to:\n\n- pear (0)\n- pear (1)\n- pear (2)\n- pear (3)\n- pear (4)\n- pear (5)\n- pear (6)\n- pear (7)\n- pear (8)\n\nThere are no new fruits or vegetables in the image beyond the pears. Each labeled item appears to be a pear, correctly matching the label. No adjustments are needed.'